In [14]:
import numpy as np
import scipy.stats
import scipy.integrate as integrate
import quadrature_zi as qs
import matplotlib.pyplot as plt
from scipy.linalg import cholesky, cho_solve, solve_triangular
from scipy.optimize import minimize
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C


In [5]:
def flambda(y):
#     return lambda x: x*0.0+2.0
#     return lambda x: scipy.stats.norm(0, 2.0).pdf(x)
#     return lambda x: np.cos(a*x+2)/(1+x**2) + 2
    return lambda x: ( (1+(x*y)**2)/(1+x**4 + y**4) ) 
# def prior(a):
#     return 0.0, 1.0
def normal(s):
    return scipy.stats.norm(0, 1.0).pdf(s)
def f(a):
    return -1.0* integrate.quad(lambda x: flambda(a)(x)*normal(x),np.NINF,np.inf)[0]
    

In [38]:
X = np.random.normal(0,1,(1000,2))
Y = np.apply_along_axis(normal, 1, X[:,1:])

In [39]:
kernel = RBF(1.0, (1e-2, 1e2))
gp = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=9)
gp.fit(X,Y)

/Library/Python/2.7/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.67092258]), 'nit': 7, 'funcalls': 56}
  " state: %s" % convergence_dict)
/Library/Python/2.7/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([-0.11355901]), 'nit': 7, 'funcalls': 73}
  " state: %s" % convergence_dict)
/Library/Python/2.7/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warnflag': 2, 'task': 'ABNORMAL_TERMINATION_IN_LNSRCH', 'grad': array([0.34903187]), 'nit': 7, 'funcalls': 91}
  " state: %s" % convergence_dict)
/Library/Python/2.7/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'warn

GaussianProcessRegressor(alpha=1e-10, copy_X_train=True,
             kernel=RBF(length_scale=1), n_restarts_optimizer=9,
             normalize_y=False, optimizer='fmin_l_bfgs_b',
             random_state=None)

In [32]:
def integrate_dim(gp, sdim,a,b, B):
    length_scale = gp.kernel_.length_scale
    A = length_scale ** 2 * np.diag(np.ones(sdim))
    I = np.identity(sdim)

    X = gp.X_train_
    n, dim = X.shape
    z = np.zeros((n, 1))
    determ = np.linalg.det(np.dot(np.linalg.inv(A), B) + I) ** (-0.5)
    abinv = np.linalg.inv(A + B)
    for i in range(n):
        x = X[i, :sdim]
        xa = X[i, sdim:]
        expon = np.exp(-0.5 * np.dot(np.dot((x - b), abinv), (x - b).T))
        factor = np.exp(-0.5 * np.sum((xa-a)**2 / (length_scale ** 2)))
        z[i, :] = factor * determ * expon

    mean = (z.T).dot(np.atleast_2d(gp.alpha_))

    return mean

In [41]:
integrate_dim(gp, 1 ,np.array([1.0]),np.array([0.0]),np.array([[1.0]]))

array([[0.24196903]])

In [22]:
Y.shape

(3000, 1)

In [25]:
X

array([[ 1.00622386,  1.47953288],
       [ 0.95017364, -1.0502642 ],
       [-0.30352868,  0.21168896],
       ...,
       [-0.91760177, -1.2320866 ],
       [ 0.70062449, -1.87709063],
       [-0.09564989,  1.95677221]])